<a href="https://colab.research.google.com/github/itonse/google-colab-archive/blob/main/goohaeyou_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 도로명 주소 변환 (서울 데이터)

import pandas as pd
import os

# 데이터 로드
file_path = "/content/juso/seoul.csv"
data = pd.read_csv(
    file_path,
    encoding='cp949',
    sep='|',
    usecols=range(13)
)

# 열 이름 설정
data.columns = ['id', 'code', '시도', 'eng_sido', '시군구', 'eng_sigungu', 'unused1', 'unused2',
                'road_code', '도로명', 'eng_roadname', 'building_main_no', 'building_sub_no']

# 문자열로 변환하고 NaN 값을 빈 문자열로 대체
data = data.fillna('').astype(str)

# 필요한 열 추출 및 주소 포맷팅
def format_address(row):
    building_sub_no = str(int(float(row['building_sub_no']))) if row['building_sub_no'] else ''
    address_parts = [row['시도'], row['시군구'], row['도로명'], building_sub_no]
    formatted_address = " ".join(part for part in address_parts if part)  # 부분이 비어있지 않으면 포함
    return formatted_address

data['FormattedAddress'] = data.apply(format_address, axis=1)

# 중복된 행 제거
data.drop_duplicates(subset='FormattedAddress', keep='first', inplace=True)

# 결과 데이터를 새로운 Excel 파일로 저장
output_path = "/content/result/seoul.xlsx"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
data.to_excel(output_path, index=False, columns=['FormattedAddress'])

<ipython-input-15-2421bba7f4d2>:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(


       id code     시도 eng_sido  시군구 eng_sigungu unused1 unused2  \
0  135965  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   
1  135964  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   
2  135964  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   
3  135965  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   
4  135962  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   

        road_code     도로명     eng_roadname building_main_no building_sub_no  \
0  116804166052.0  개포로17길  Gaepo-ro 17-gil              0.0             9.0   
1  116804166053.0  개포로20길  Gaepo-ro 20-gil              0.0            25.0   
2  116803121022.0     논현로      Nonhyeon-ro              0.0            88.0   
3  116803122001.0     개포로         Gaepo-ro              0.0           251.0   
4  116803121022.0     논현로      Nonhyeon-ro              0.0            42.0   

      FormattedAddress  
0   서울특별시 강남구 개포로17길 9  
1  서울특별시 강남구 개포로20길 25  
2     서울특별시 강남구 논현로 88  
3    서울특별시 강남구 개포로 251

In [3]:
## 지역별 데이터를 무작위로 섞음

import pandas as pd
import numpy as np

# 파일 경로 설정
excel_path = 'result/location.xlsx'

# 엑셀 파일 로드
df = pd.read_excel(excel_path, header=None)
df.columns = ['ShuffledAddress']

# 데이터프레임의 행을 무작위로 섞음
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 섞은 데이터 저장
output_path = 'result/shuffled_location.xlsx'
shuffled_df.to_excel(output_path, index=False)

# 샘플로 만들어진 데이터 5개 출력
print(shuffled_df.head())

              ShuffledAddress
0         서울 관악구 남부순환로230길 86
1         경북 경주시 강동면 수궁서당길 49
2  강원특별자치도 삼척시 미로면 강원남부로 4167
3              대구 남구 양지로8길 68
4      경기 용인시 처인구 원설로490번길 28


In [5]:
## member 데이터 10만개 bulk insert문 생성

import pandas as pd
import random
import string
import nltk
from datetime import datetime, timedelta
from nltk.corpus import words

# location 데이터 로드
locations_df = pd.read_excel('result/shuffled_location.xlsx')

# 파일 저장 경로 설정
output_sql_path = 'result/bulk_insert_member.sql'

# NLTK 단어 목록 다운로드 및 필터
nltk.download('words')
word_list = words.words()
# 영문자 단어만 필터링 후 4,5,6 자리 단어만 추출
four_letter_words = [word for word in word_list if len(word) == 4 and word.isalpha() and word == word.lower()]
five_letter_words = [word for word in word_list if len(word) == 5 and word.isalpha() and word == word.lower()]

# 한국인 성 리스트
korean_surnames = ["김", "이", "박", "최", "정", "강", "조", "윤", "장", "임", "오", "한", "신", "서", "권", "황", "안", "송", "류", "전"]

# 남성 이름 리스트
korean_male_given_names = [
    "민준", "서준", "예준", "도윤", "시우", "주원", "하준", "지호", "지후", "준서",
    "준우", "현우", "도현", "지훈", "건우", "우진", "선우", "서진", "민재", "현준"
]

# 여성 이름 리스트
korean_female_given_names = [
    "서연", "민서", "지우", "서현", "민지", "지유", "지민", "채원", "수아", "다은",
    "예은", "서아", "수민", "서윤", "지안", "하윤", "유진", "은서", "지아", "소율"
]

# 지역코드 매핑
region_code_map = {
    "서울": 101,
    "경기": 102,
    "부산": 103,
    "경남": 104,
    "인천": 105,
    "경북": 106,
    "대구": 107,
    "충남": 108,
    "전남": 109,
    "전북특별자치도": 110,
    "충북": 111,
    "강원특별자치도": 112,
    "대전": 113,
    "광주": 114,
    "울산": 115,
    "제주특별자치도": 116,
    "세종특별자치시": 117
}

# 날짜 범위 지정
start_date = datetime(2024, 7, 1)
end_date = datetime(2024, 7, 27)

# 비밀번호: '11111111' 해싱
hashed_password = '$2a$10$JMUdQsovXBtYU7FblqgyzOn6h0bE5.rzdMOMZSpuewiM1b.YCFcRm'

# 유니크한 username을 저장하기 위한 집합
username_set = set()

# 랜덤 시간 생성
def generate_weighted_random_time():
    """공고 작성 시간 비중: 오전 9시 ~ 12시에 60%, 12시 ~ 15시에 20%, 나머지 시간대에 20%"""
    rnd = random.random()
    if rnd < 0.6:
        # 오전 9시 ~ 12시 (60%)
        hour = random.randint(9, 11)
    elif rnd < 0.8:
        # 오후 12시 ~ 15시 (20%)
        hour = random.randint(12, 14)
    else:
        # 나머지 시간대 (20%)
        hour = random.choice([*range(0, 9), *range(15, 24)])

    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    microsecond = random.randint(0, 999999)

    return timedelta(hours=hour, minutes=minute, seconds=second, microseconds=microsecond)

# 랜덤 날짜 생성
def generate_random_date(start, end):
    total_seconds = random.randint(0, int((end - start).total_seconds()))
    return start + timedelta(seconds=total_seconds)

# 데이터 수 설정
num_samples = 100000   # 10만개 설정

members = []
for i in range(num_samples):
    id = i + 1
    # 랜덤 date_time 생성
    random_date_time = generate_random_date(start_date, end_date) + generate_weighted_random_time()
    created_at = modified_at = random_date_time.strftime('%Y-%m-%d %H:%M:%S.%f')

    birth_year = random.randint(1964, 2004)
    birth = f"{birth_year}-{'%02d' % random.randint(1, 12)}-{'%02d' % random.randint(1, 28)}"
    gender = random.choice(['MALE', 'FEMALE'])
    location = locations_df.iloc[i, 0]
    surname = random.choice(korean_surnames)
    given_name = random.choice(korean_male_given_names if gender == 'MALE' else korean_female_given_names)
    name = surname + given_name
    phone_number = "010" + ''.join(random.choices(string.digits, k=8))
    # 지역코드 추출
    region = location.split()[0]  # 도로명 주소에서 첫 번째 단어는 시,도
    region_code = region_code_map.get(region, 999)

    # 유니크한 랜덤 username 생성
    while True:
        if random.choice([True, False, False]):  # 5자리 영단어 + 2자리 숫자 (1/3 확률)
            username = random.choice(five_letter_words) + ''.join(random.choices(string.digits, k=2))
        elif random.choice([True, False]):  # 4자리 영단어 + 3자리 숫자 (나머지 2/3 확률 중 1/2 확률)
            username = random.choice(four_letter_words) + ''.join(random.choices(string.digits, k=3))
        else:  # 5자리 영단어 + 2자리 영소문자 (나머지 1/3 확률)
            username = random.choice(five_letter_words) + ''.join(random.choices(string.ascii_lowercase, k=2))

        # 중복되지 않은 경우 루프 탈출
        if username not in username_set:
            username_set.add(username)
            break

    # 랜덤 email 생성
    email = username + "@goohaeyou.com"

    member = (id, created_at, modified_at, birth, email, gender, location, name, hashed_password, phone_number, username, region_code)
    members.append(member)

# SQL 파일 생성
with open(output_sql_path, 'w') as file:
    file.write("INSERT INTO member (id, created_at, modified_at, birth, email, gender, location, name, password, phone_number, username, authenticated, rest_cash, role, transaction_count, region_code) VALUES\n")
    for i, member in enumerate(members):
        sql = f"('{member[0]}', '{member[1]}', '{member[2]}', '{member[3]}', '{member[4]}', '{member[5]}', '{member[6]}', '{member[7]}', '{member[8]}', '{member[9]}', '{member[10]}', 1, 0, 'USER', 0, '{member[11]}')"
        if i < len(members) - 1:
            sql += ",\n"
        else:
            sql += ";\n"
        file.write(sql)

        # 샘플로 상위 5개의 SQL 문을 출력
        if i < 5:
            print(sql)

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


('1', '2024-07-19 15:05:38.677739', '2024-07-19 15:05:38.677739', '1971-05-24', 'echo008@goohaeyou.com', 'FEMALE', '서울 관악구 남부순환로230길 86', '장채원', '$2a$10$JMUdQsovXBtYU7FblqgyzOn6h0bE5.rzdMOMZSpuewiM1b.YCFcRm', '01090787118', 'echo008', 1, 0, 'USER', 0, '101'),

('2', '2024-07-13 06:09:55.088463', '2024-07-13 06:09:55.088463', '1965-06-04', 'lout400@goohaeyou.com', 'MALE', '경북 경주시 강동면 수궁서당길 49', '전준우', '$2a$10$JMUdQsovXBtYU7FblqgyzOn6h0bE5.rzdMOMZSpuewiM1b.YCFcRm', '01045847900', 'lout400', 1, 0, 'USER', 0, '106'),

('3', '2024-07-13 11:18:16.258205', '2024-07-13 11:18:16.258205', '1991-01-28', 'diose45@goohaeyou.com', 'FEMALE', '강원특별자치도 삼척시 미로면 강원남부로 4167', '송예은', '$2a$10$JMUdQsovXBtYU7FblqgyzOn6h0bE5.rzdMOMZSpuewiM1b.YCFcRm', '01019265435', 'diose45', 1, 0, 'USER', 0, '112'),

('4', '2024-07-19 13:55:13.429055', '2024-07-19 13:55:13.429055', '1985-01-04', 'niece56@goohaeyou.com', 'MALE', '대구 남구 양지로8길 68', '최시우', '$2a$10$JMUdQsovXBtYU7FblqgyzOn6h0bE5.rzdMOMZSpuewiM1b.YCFcRm', '010701725

In [ ]:
## username 목록 생성

import re
import pandas as pd

# 파일 경로 설정
input_sql_path = 'result/bulk_insert_member.sql'
output_ids_usernames_path = 'result/ids_usernames.csv'

# SQL 파일에서 id와 username 값을 추출하여 목록 생성
ids_usernames = []
with open(input_sql_path, 'r') as file:
    lines = file.readlines()
    for line in lines[1:]:  # 첫 줄은 INSERT INTO statement
        if line.strip().endswith(';'):
            line = line.strip()[:-1]  # 마지막 세미콜론 제거
        # 정규식을 사용하여 id와 username 추출
        match = re.search(r"\('(\d+)', '[^']*', '[^']*', '[^']*', '[^']*', '[^']*', '[^']*', '[^']*', '[^']*', '[^']*', '([^']*)'", line)
        if match:
            id_value = match.group(1)
            username_value = match.group(2)
            ids_usernames.append((id_value, username_value))

# id와 username 값을 CSV 파일로 저장
df_ids_usernames = pd.DataFrame(ids_usernames, columns=['id', 'username'])
df_ids_usernames.to_csv(output_ids_usernames_path, index=False)

[('1', 'cent990'), ('2', 'gregowx'), ('3', 'slipe59'), ('4', 'locum05'), ('5', 'sifacoi'), ('6', 'fudge77'), ('7', 'rase779'), ('8', 'fogleli'), ('9', 'heave23'), ('10', 'targefp')]


In [60]:
## title(공고 제목), body(공고 내용) 쌍의 데이터 100만개 생성

import pandas as pd
import random

def generate_dataset(num_samples):
    descriptors = [
    "단순", "[바로가능]", "오늘", "지금", "근처 사시는 분", "하루만", "[30분 소요]",
    "경험자 우대!", "상시", "내일", "[긴급]", "[협의 가능]", "쉬워요/", "1시간 이하/",
    "[10분컷]", "[간단]"
    ]
    job_types = [
    # 일상 도움
    "강아지 산책", "강아지 발톱 정리", "고양이 목욕", "햄스터 집 청소",
    "화분 분갈이", "커튼봉 설치", "여행계획 세우기", "기차 예매", "쇼핑 같이",
    "전화 문의 대신", "찢어진 인형 수리", "그늘막 설치", "모루인형 만들기",
    "시험장까지 픽업", "아이 픽업", "수강 신청",
    # 정리 및 청소
    "정리정돈", "집 청소", "화단 청소", "이불 빨래", "베란다 창문 닦기",
    "대형 폐기물 운반", "현수막 수거", "가구 재배치", "옷장 정리", "자동차 내부 청소",
    "창고 정리", "창문 틈새 청소", "냉장고 정리",
    # 물류 및 배송
    "배달", "짐 이동", "우편물", "헬스기구 운반", "가게에서 물건 픽업",
    "터미널 수화물 픽업", "굿즈 대리구매", "팝업스토어 대리구매", "과일포장 작업",
    "식품 포장", "준비물 전달",
    # 기술 작업
    "침대 프레임 조립", "커튼봉 설치", "싱크대 수전 교체", "집 벽지 붙이기", "차량 와이퍼 교체",
    "컴퓨터 설치", "노트북 수리", "증명사진 보정", "가구 조립", "조명 설치",
    # 매장
    "매장 오픈 준비", "가게 오픈 준비", "식당 오픈 준비", "어플 리뷰 관리", "매장 내부 장식",
    # 사무 및 교육
    "미술학원 일일 보조 강사", "학원 수강생 출석 체크", "수업 자료 정리", "시험 감독 보조",
    "과제 준비 조보", "서류 정리", "강의 노트 정리",
    # 행사
    "체육대회 정리", "스타벅스 프리퀀시 수령 오픈런", "이벤트 장식 설치", "행사 참여 인원 관리",
    "행사 안내", "기념품 배포", "체험 행사 보조",
    # 기타
    "단순조립", "헤어 모델", "고양이 방문 시터", "피아노 운반/조율", "유튜브 콘텐츠 촬영",
    "벌집 제거", "운동 계획 세우기"
    ]
    actions = [
        "해주실분", "해주세요", "해주실 분 구합니다", "해주실 분 구해요", "하실 분 구해요", "하실 분 있으신가요?", "도와주실 분 찾습니다",
        "해주실분 찾아요", "도와주실 분 구해요", "하실 분?", "급하게 구해요"
    ]
    details1 = [
        "경험 있으신 분", "성실히 해주실 분", "약속한 시간 내에 끝내실 수 있는 분", "정해진 시간 내에 끝내실 수 있는 분",
        "해주실 분 찾아요." "잘 하시는 분 찾아요.", "많이 해보신 분 찾습니다."
    ]
    details2 = [
        "1명만 모집합니다", "오래 하실 수 있는 분 좋아요", "자주 하실 수 있으면 좋습니다", "댓글로 문의 해주세요~",
        "승인되시면 전화로 안내드립니다", "많이 지원해주세요", "자신 있는 분 지원하세요", "책임감 있으신 분 환영해요.",
        "현장에서 바로 지급합니다", "댓글에 질문 남겨주세요.", "승인 후 채팅 답장 부탁합니다.", "채팅으로 자세한 시간약속 잡을게요",
        "책임감있게 해주실 분 환영해요", "금액 제시도 가능합니다", "성격이 밝으신 분 환영", "초보자는 지원하지 말아주세요."
        "시간 약속 잘 지키시는 분 이였으면 좋겠어요.", "시간 조절 가능합니다.", "시간/페이 협의 가능", "일찍 끝나도 약속한대로 지급 해드려요"
    ]

    selected_job_types = random.choices(job_types, k=num_samples)

    titles = [f"{random.choice(descriptors)} {job_type} {random.choice(actions)}"
              for job_type in selected_job_types]
    bodies = [f"{job_type} {random.choice(details1)} \n{random.choice(details2)}"
              for job_type in selected_job_types]

    # 데이터셋 생성
    data = pd.DataFrame({
        "title": titles,
        "body": bodies,
    })

    return data

num_samples = 1000000
full_dataset = generate_dataset(num_samples)

sava_path = "result/title_body.xlsx"
full_dataset.to_excel(sava_path, index=False)

# 샘플로 만들어진 title-body 데이터 5쌍 출력
print(full_dataset.head())

                        title  \
0  [30분 소요] 강의 노트 정리 해주실분 찾아요   
1    [10분컷] 헤어 모델 하실 분 있으신가요?   
2              상시 배달 하실 분 구해요   
3             단순 강아지 산책 하실 분?   
4    상시 미술학원 일일 보조 강사 급하게 구해요   

                                               body  
0            강의 노트 정리 많이 해보신 분 찾습니다. \n성격이 밝으신 분 환영  
1  헤어 모델 해주실 분 찾아요.잘 하시는 분 찾아요. \n승인 후 채팅 답장 부탁합니다.  
2               배달 성실히 해주실 분 \n일찍 끝나도 약속한대로 지급 해드려요  
3                    강아지 산책 경험 있으신 분 \n성격이 밝으신 분 환영  
4        미술학원 일일 보조 강사 성실히 해주실 분 \n책임감있게 해주실 분 환영해요  


In [61]:
## job_post 데이터 100만개 bulk insert문 생성

import pandas as pd
import random
from datetime import datetime, date, timedelta

# 파일 경로
input_excel_title_body_path = 'result/title_body.xlsx'
input_excel_location_path = 'result/shuffled_location.xlsx'
output_path = 'result/bulk_insert_job_post.sql'

# 파일 로드
title_body_df = pd.read_excel(input_excel_title_body_path)
locations_df = pd.read_excel(input_excel_location_path)

# 지역코드 매핑
region_code_map = {
    "서울": 101,
    "경기": 102,
    "부산": 103,
    "경남": 104,
    "인천": 105,
    "경북": 106,
    "대구": 107,
    "충남": 108,
    "전남": 109,
    "전북특별자치도": 110,
    "충북": 111,
    "강원특별자치도": 112,
    "대전": 113,
    "광주": 114,
    "울산": 115,
    "제주특별자치도": 116,
    "세종특별자치시": 117
}

# 날짜 범위 지정
start_date = datetime(2024, 7, 30)
end_date = datetime(2024, 8, 13)

# 데이터 생성
application_count = 0
comments_count = 0
increment_view_count = 0
interests_count = 0
closed = False
employed = False

# 랜덤 시간 생성
def generate_weighted_random_time():
    """공고 작성 시간 비중: 오전 9시 ~ 12시에 60%, 12시 ~ 15시에 20%, 나머지 시간대에 20%"""
    rnd = random.random()
    if rnd < 0.6:
        # 오전 9시 ~ 12시 (60%)
        hour = random.randint(9, 11)
    elif rnd < 0.8:
        # 오후 12시 ~ 15시 (20%)
        hour = random.randint(12, 14)
    else:
        # 나머지 시간대 (20%)
        hour = random.choice([*range(0, 9), *range(15, 24)])

    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    microsecond = random.randint(0, 999999)

    return timedelta(hours=hour, minutes=minute, seconds=second, microseconds=microsecond)

# 랜덤 날짜 생성
def generate_random_date(start, end):
    total_seconds = random.randint(0, int((end - start).total_seconds()))
    return start + timedelta(seconds=total_seconds)

# 데이터 개수 지정
num_samples = 1000000

# SQL 파일 오픈
with open(output_path, 'w') as file:
    # Bulk insert 시작 부분
    file.write("INSERT INTO job_post (id, created_at, modified_at, application_count, closed, comments_count, deadline, employed, increment_view_count, interests_count, job_start_date, location, title, member_id, region_code) VALUES\n")


    for i in range(0, num_samples):
        # 랜덤 created_at date_time 생성
        random_date_time = generate_random_date(start_date, end_date) + generate_weighted_random_time()
        created_at = modified_at = random_date_time

        deadline = random_date_time + timedelta(days=7)
        job_start_date = deadline + timedelta(days=1)

        title = title_body_df.iloc[i, 0]
        location = locations_df.iloc[i, 0]

        # 지역코드 추출
        region = location.split()[0]  # 도로명 주소에서 첫 번째 단어는 시,도
        region_code = region_code_map.get(region, 999)

        # member_id를 1에서 100000 사이의 무작위 값으로 설정
        member_id = random.randint(1, 100000)

        # 데이터 값을 생성
        sql = f"({i + 1}, '{created_at.strftime('%Y-%m-%d %H:%M:%S.%f')}', '{modified_at.strftime('%Y-%m-%d %H:%M:%S.%f')}', {application_count}, {int(closed)}, {comments_count}, '{deadline.strftime('%Y-%m-%d')}', {int(employed)}, {increment_view_count}, {interests_count}, '{job_start_date.strftime('%Y-%m-%d')}', '{location}', '{title}', {member_id}, {region_code})"

        # 마지막 데이터가 아니면 쉼표와 줄바꿈 추가
        if i < num_samples - 1:
            sql += ",\n"
        else:
            sql += ";\n"

        file.write(sql)

        # 샘플로 상위 5개의 SQL문 출력
        if i < 5:
            print(sql)

(1, '2024-08-07 06:26:59.385080', '2024-08-07 06:26:59.385080', 0, 0, 0, '2024-08-14', 0, 0, 0, '2024-08-15', '서울 관악구 남부순환로230길 86', '[30분 소요] 강의 노트 정리 해주실분 찾아요', 97159, 101),

(2, '2024-08-04 12:27:10.533912', '2024-08-04 12:27:10.533912', 0, 0, 0, '2024-08-11', 0, 0, 0, '2024-08-12', '경북 경주시 강동면 수궁서당길 49', '[10분컷] 헤어 모델 하실 분 있으신가요?', 25171, 106),

(3, '2024-08-08 13:58:22.688356', '2024-08-08 13:58:22.688356', 0, 0, 0, '2024-08-15', 0, 0, 0, '2024-08-16', '강원특별자치도 삼척시 미로면 강원남부로 4167', '상시 배달 하실 분 구해요', 26800, 112),

(4, '2024-08-10 17:52:14.388073', '2024-08-10 17:52:14.388073', 0, 0, 0, '2024-08-17', 0, 0, 0, '2024-08-18', '대구 남구 양지로8길 68', '단순 강아지 산책 하실 분?', 68705, 107),

(5, '2024-08-03 20:20:19.955888', '2024-08-03 20:20:19.955888', 0, 0, 0, '2024-08-10', 0, 0, 0, '2024-08-11', '경기 용인시 처인구 원설로490번길 28', '상시 미술학원 일일 보조 강사 급하게 구해요', 47460, 102),



In [66]:
## job_post_detail 데이터 100만개 bulk insert문 생성

import re
import pandas as pd

# 파일 경로
excel_title_body_path = 'result/title_body.xlsx'
ids_usernames_path = 'result/ids_usernames.csv'
input_job_post_path = 'result/bulk_insert_job_post.sql'
output_path = 'result/bulk_insert_job_post_detail.sql'

# 파일 로드
title_body_df = pd.read_excel(excel_title_body_path)
ids_usernames_df = pd.read_csv(ids_usernames_path)

# ids_usernames_df를 딕셔너리로 변환
id_to_username = dict(zip(ids_usernames_df['id'], ids_usernames_df['username']))

# bulk_insert_job_post.sql 파일에서 job_post_id, created_at, member_id 추출
job_post_ids_created_at_members = []

with open(input_job_post_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        # 정규식 수정: job_post_id, created_at, member_id를 추출
        match = re.match(r"\((\d+),\s*'([^']*)',.*?,\s*(\d+),\s*\d+\)", line.strip())
        if match:
            job_post_id = int(match.group(1))
            created_at_str = match.group(2)
            member_id = int(match.group(3))
            job_post_ids_created_at_members.append((job_post_id, created_at_str, member_id))

# SQL 파일 작성
with open(output_path, 'w') as file:
    # Bulk insert 시작 부분
    file.write("INSERT INTO job_post_detail (id, created_at, modified_at, author, body, job_post_id) VALUES\n")

    for i in range(len(job_post_ids_created_at_members)):
        job_post_id, created_at_str, member_id = job_post_ids_created_at_members[i]
        modified_at_str = created_at_str
        body = title_body_df.iloc[i, 1]

        # member_id에 해당하는 author 추출
        author = id_to_username.get(member_id, 'unknown')

        # 데이터 값을 생성
        sql = f"({i + 1}, '{created_at_str}', '{modified_at_str}', '{author}', '{body}', {job_post_id})"

        # 마지막 데이터가 아니면 쉼표와 줄바꿈 추가
        if i < len(job_post_ids_created_at_members) - 1:
            sql += ",\n"
        else:
            sql += ";\n"

        file.write(sql)

        # 샘플로 상위 5개의 SQL문 출력
        if i < 5:
            print(sql)

(1, '2024-08-07 06:26:59.385080', '2024-08-07 06:26:59.385080', 'aper424', '강의 노트 정리 많이 해보신 분 찾습니다. 
성격이 밝으신 분 환영', 1),

(2, '2024-08-04 12:27:10.533912', '2024-08-04 12:27:10.533912', 'orate06', '헤어 모델 해주실 분 찾아요.잘 하시는 분 찾아요. 
승인 후 채팅 답장 부탁합니다.', 2),

(3, '2024-08-08 13:58:22.688356', '2024-08-08 13:58:22.688356', 'genos14', '배달 성실히 해주실 분 
일찍 끝나도 약속한대로 지급 해드려요', 3),

(4, '2024-08-10 17:52:14.388073', '2024-08-10 17:52:14.388073', 'laterqo', '강아지 산책 경험 있으신 분 
성격이 밝으신 분 환영', 4),

(5, '2024-08-03 20:20:19.955888', '2024-08-03 20:20:19.955888', 'scutaxy', '미술학원 일일 보조 강사 성실히 해주실 분 
책임감있게 해주실 분 환영해요', 5),



In [89]:
## essential 데이터 100만개 bulk insert문 생성

import random

# 파일 경로 설정
output_sql_path = 'result/bulk_insert_essential.sql'

# 데이터 범위 및 옵션 설정
num_samples = 1000000
genders = ['UNDEFINED'] * 60 + ['MALE'] * 20 + ['FEMALE'] * 20  # 무관6, 남2, 여2 비율로 설정
age_distribution = [0] * 40 + [20] * 30 + [25] * 5 + [30] * 10 + [40] * 5  # 해당 비율로 나이 분포 설정

# SQL 파일 생성
with open(output_sql_path, 'w') as file:
    # Bulk insert 시작 부분
    file.write("INSERT INTO essential (id, gender, min_age, job_post_detail_id) VALUES\n")

    for i in range(num_samples):
        gender = random.choice(genders)  # 랜덤으로 성별 선택
        min_age = random.choice(age_distribution)  # 랜덤으로 나이 선택
        job_post_detail_id = i + 1  # 1부터 100만까지 순차적 증가

        # 데이터 값을 생성
        sql = f"({i + 1}, '{gender}', {min_age}, {job_post_detail_id})"

        # 마지막 데이터가 아니면 쉼표와 줄바꿈 추가
        if i < num_samples:
            sql += ",\n"
        else:
            sql += ";\n"

        file.write(sql)

        # 샘플로 상위 5개의 SQL문 출력
        if i < 5:
            print(sql)

(1, 'MALE', 25, 1),

(2, 'UNDEFINED', 0, 2),

(3, 'UNDEFINED', 30, 3),

(4, 'UNDEFINED', 0, 4),

(5, 'FEMALE', 20, 5),



In [90]:
## wage 데이터 100만개 bulk insert문 생성

import random

# SQL 파일에서 데이터 읽기
def read_sql_file(file_path):
    titles = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            if line.strip().endswith('),') or line.strip().endswith(');'):  # 데이터가 있는 행인지 확인
                parts = line.split(',')
                title = parts[12].strip().strip("'")  # 'title' 열의 값을 추출
                titles.append(title)
    return titles

# wage 데이터 생성
def generate_wage(title, index):
    day_keywords = ["오늘", "지금", "하루만", "내일", "체육대회 정리", "헬스기구 운반", "일일", "그늘막 설치",
                    "집 청소", "화단 청소", "이불 빨래", "베란다 창문 닦기", "가구 재배치", "옷장 정리",
                    "자동차 내부 청소", "창문 틈새 청소", "냉장고 정리", "굿즈 대리구매", "팝업스토어 대리구매",
                    "침대 프레임 조립", "컴퓨터 설치", "노트북 조립", "증명사진 보정", "벌집 제거", "가구 조립",
                    "조명 설치", "프리퀀시"]
    hour_keywords = ["30분", "1시간", "10분컷", "커튼봉 설치", "기차 예매", "고양이 목욕",
                     "강아지 발톱 정리", "햄스터 집 청소", "시험장까지 픽업"]

    # pay_basis 설정
    if any(keyword in title for keyword in day_keywords):
        pay_basis = 'TOTAL_DAYS'
        work_days = 1
        work_time = 0
    elif any(keyword in title for keyword in hour_keywords):
        pay_basis = 'TOTAL_HOURS'
        work_days = 1
        work_time = 1
    else:
        pay_basis = random.choice(['TOTAL_HOURS'] * 70 + ['TOTAL_DAYS'] * 30)
        if pay_basis == 'TOTAL_HOURS':
            work_days = 1
            work_time = random.choice([1, 2] * 70 + [3, 4, 5] * 20 + [6, 7, 8] * 10)  # 1,2는 70%, 3~5는 20%, 6~8는 10% 비율로 설정
        else:
            work_days = random.choice([1, 2, 3] * 70 + [4, 5] * 25 + [6, 7] * 5)  # 1~3은 70%, 4,5는 25%, 6,7은 5% 비율로 설정
            work_time = 0

    # cost 설정
    if pay_basis == 'TOTAL_HOURS':
        if work_time in [1, 2]:
            cost = random.choice(range(10000, 35000, 5000))  # 10000에서 30000까지, 5000 단위
        elif work_time in [3, 4, 5]:
            cost = random.choice(range(20000, 45000, 5000))  # 20000에서 40000까지, 5000 단위
        else:
            cost = random.choice(range(40000, 65000, 5000))  # 40000에서 60000까지, 5000 단위
    else:
        if work_days in [1, 2, 3]:
            cost = random.choice(range(20000, 45000, 5000))  # 20000에서 40000까지, 5000 단위
        elif work_days in [4, 5]:
            cost = random.choice(range(30000, 65000, 5000))  # 30000에서 60000까지, 5000 단위
        else:
            cost = random.choice(range(40000, 95000, 5000))  # 40000에서 90000까지, 5000 단위

    wage_payment_method = random.choice(['INDIVIDUAL_PAYMENT', 'SERVICE_PAYMENT'])
    paid = 0  # 모두 false니까 0으로 설정

    return f"({index + 1}, {cost}, {paid}, '{pay_basis}', '{wage_payment_method}', {work_days}, {work_time}, {index + 1}),\n"

# SQL 파일 쓰기
def write_to_sql_file(titles, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write("INSERT INTO wage (id, cost, paid, pay_basis, wage_payment_method, work_days, work_time, job_post_detail_id) VALUES\n")
        for index, title in enumerate(titles):
            sql_statement = generate_wage(title, index)
            file.write(sql_statement)
            if index < 5:
                print(sql_statement, end='')  # 샘플로 상위 5개의 SQL문 출력

        # 마지막 콤마 제거 및 쿼리 종료 처리
        file.seek(0, 2)  # 파일의 끝으로 이동
        file.truncate(file.tell() - 2)  # 마지막 콤마 삭제
        file.write(";\n")  # 쿼리 종료

# 파일 경로 설정
input_file_path = 'result/bulk_insert_job_post.sql'
output_file_path = 'result/bulk_insert_wage.sql'

# 파일 읽고 쓰기
job_post_titles = read_sql_file(input_file_path)
write_to_sql_file(job_post_titles, output_file_path)

(1, 25000, 0, 'TOTAL_HOURS', 'INDIVIDUAL_PAYMENT', 1, 1, 1),
(2, 10000, 0, 'TOTAL_HOURS', 'SERVICE_PAYMENT', 1, 1, 2),
(3, 55000, 0, 'TOTAL_HOURS', 'INDIVIDUAL_PAYMENT', 1, 7, 3),
(4, 30000, 0, 'TOTAL_DAYS', 'SERVICE_PAYMENT', 1, 0, 4),
(5, 35000, 0, 'TOTAL_DAYS', 'INDIVIDUAL_PAYMENT', 1, 0, 5),


In [92]:
## job_post_category 데이터 200만개 bulk insert문 생성

import pandas as pd
import csv

# 카테고리 데이터 정의
category_data = [
    (1, True, 1, '업무', None, 'TASK'),
    (2, True, 1, '지역', None, 'REGION'),
    (3, True, 2, '일상 도움', 1, 'TASK'),
    (4, True, 2, '정리 및 청소', 1, 'TASK'),
    (5, True, 2, '물류 및 배송', 1, 'TASK'),
    (6, True, 2, '기술 작업', 1, 'TASK'),
    (7, True, 2, '매장', 1, 'TASK'),
    (8, True, 2, '사무 및 교육', 1, 'TASK'),
    (9, True, 2, '행사', 1, 'TASK'),
    (10, True, 2, '기타', 1, 'TASK'),
    (11, True, 2, '서울', 2, 'REGION'),
    (12, True, 2, '경기', 2, 'REGION'),
    (13, True, 2, '부산', 2, 'REGION'),
    (14, True, 2, '경남', 2, 'REGION'),
    (15, True, 2, '인천', 2, 'REGION'),
    (16, True, 2, '경북', 2, 'REGION'),
    (17, True, 2, '대구', 2, 'REGION'),
    (18, True, 2, '충남', 2, 'REGION'),
    (19, True, 2, '전남', 2, 'REGION'),
    (20, True, 2, '전북특별자치도', 2, 'REGION'),
    (21, True, 2, '충북', 2, 'REGION'),
    (22, True, 2, '강원특별자치도', 2, 'REGION'),
    (23, True, 2, '대전', 2, 'REGION'),
    (24, True, 2, '광주', 2, 'REGION'),
    (25, True, 2, '울산', 2, 'REGION'),
    (26, True, 2, '제주특별자치도', 2, 'REGION'),
    (27, True, 2, '세종특별자치시', 2, 'REGION')
]

# DataFrame으로 변환
category_df = pd.DataFrame(category_data, columns=['id', 'enabled', 'level', 'name', 'parent_id', 'type'])

# 지역코드 매핑
region_code_map = {
    "서울": 101,
    "경기": 102,
    "부산": 103,
    "경남": 104,
    "인천": 105,
    "경북": 106,
    "대구": 107,
    "충남": 108,
    "전남": 109,
    "전북특별자치도": 110,
    "충북": 111,
    "강원특별자치도": 112,
    "대전": 113,
    "광주": 114,
    "울산": 115,
    "제주특별자치도": 116,
    "세종특별자치시": 117
}

# job_types 정의
job_types = {
    "일상 도움": ["강아지 산책", "강아지 발톱 정리", "고양이 목욕", "햄스터 집 청소", "화분 분갈이", "커튼봉 설치", "여행계획 세우기", "기차 예매", "쇼핑 같이", "전화 문의 대신", "찢어진 인형 수리", "그늘막 설치", "모루인형 만들기", "시험장까지 픽업", "아이 픽업", "수강 신청"],
    "정리 및 청소": ["정리정돈", "집 청소", "화단 청소", "이불 빨래", "베란다 창문 닦기", "대형 폐기물 운반", "현수막 수거", "가구 재배치", "옷장 정리", "자동차 내부 청소", "창고 정리", "창문 틈새 청소", "냉장고 정리"],
    "물류 및 배송": ["배달", "짐 이동", "우편물", "헬스기구 운반", "가게에서 물건 픽업", "터미널 수화물 픽업", "굿즈 대리구매", "팝업스토어 대리구매", "과일포장 작업", "식품 포장", "준비물 전달"],
    "기술 작업": ["침대 프레임 조립", "커튼봉 설치", "싱크대 수전 교체", "집 벽지 붙이기", "차량 와이퍼 교체", "컴퓨터 설치", "노트북 수리", "증명사진 보정", "가구 조립", "조명 설치"],
    "매장": ["매장 오픈 준비", "가게 오픈 준비", "식당 오픈 준비", "어플 리뷰 관리", "매장 내부 장식"],
    "사무 및 교육": ["미술학원 일일 보조 강사", "학원 수강생 출석 체크", "수업 자료 정리", "시험 감독 보조", "과제 준비 조보", "서류 정리", "강의 노트 정리"],
    "행사": ["체육대회 정리", "스타벅스 프리퀀시 수령 오픈런", "이벤트 장식 설치", "행사 참여 인원 관리", "행사 안내", "기념품 배포", "체험 행사 보조"],
    "기타": ["단순조립", "헤어 모델", "고양이 방문 시터", "피아노 운반/조율", "유튜브 콘텐츠 촬영", "벌집 제거", "운동 계획 세우기"]
}

# job_post_id와 title, location, region_code 추출
job_posts = []

with open('result/bulk_insert_job_post.sql', 'r') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line.startswith('INSERT INTO') or not line:
            continue
        if line.endswith('),'):
            line = line[:-1]  # ), 행의 마지막에 있는 쉼표 제거
        line = line.strip('()')
        parts = list(csv.reader([line], quotechar="'", skipinitialspace=True))[0]

        job_post_id = int(parts[0])
        title = parts[12]
        location = parts[11]
        region_code = int(parts[14].strip(");"))  # 문자열에서 )와 ;를 제거
        job_posts.append((job_post_id, title, location, region_code))

with open('result/bulk_insert_job_post_category.sql', 'w') as file:
    # Bulk insert 시작 부분
    file.write("INSERT INTO job_post_category (id, category_id, job_post_id) VALUES\n")

    entry_id = 1
    values = []

    for job_post_id, title, location, region_code in job_posts:
        # title에서 키워드가 포함된 TASK 카테고리 선택
        task_category_added = False
        for category, keywords in job_types.items():
            if any(keyword in title for keyword in keywords):
                task_category_id = category_df[category_df['name'] == category].iloc[0]['id']
                values.append(f"({entry_id}, {task_category_id}, {job_post_id})")
                entry_id += 1
                task_category_added = True
                break

        if not task_category_added:
            # 키워드가 없는 경우 TASK 카테고리를 '기타'로 설정
            task_category_id = category_df[category_df['name'] == '기타'].iloc[0]['id']
            values.append(f"({entry_id}, {task_category_id}, {job_post_id})")
            entry_id += 1

        # REGION 카테고리 추가
        for region, code in region_code_map.items():
            if code == region_code:
                region_name = region
                region_category_id = category_df[category_df['name'] == region_name].iloc[0]['id']
                values.append(f"({entry_id}, {region_category_id}, {job_post_id})")
                entry_id += 1
                break

    file.write(",\n".join(values) + ";\n")

    # 샘플로 상위 5개의 SQL문 출력
    for line in values[:5]:
        print(line)

(1, 8, 1)
(2, 11, 1)
(3, 10, 2)
(4, 16, 2)
(5, 5, 3)
